In [1]:
from string import *
from sys import argv
from Bio import File
from Bio import SeqIO
import sys
import tempfile
import os
from os import system
import subprocess

In [9]:
NN_bin = './disembl'
SG_bin = './sav_gol'

def JensenNet(sequence):
    outFile = tempfile.mktemp()
    inFile= tempfile.mktemp()
    
    open(inFile,'w').write(sequence+'\n')
    #inFile = 'OC1.txt'
    #outFile ='OC2.txt'
    system(NN_bin + '< ' + inFile +' >' + outFile)
    REM465 = []
    COILS = []
    HOTLOOPS = []
    resultsFile = open(outFile,'r')
    results = resultsFile.readlines()
    resultsFile.close()
    for result in results:
        result =result.split()
        coil = round(float(result[0]),6)
        COILS.append(coil)
        hotloop = round(float(result[1]),6)
        HOTLOOPS.append(hotloop)
        rem465 = round(float(result[2]),6)
        REM465.append(rem465)
    os.remove(inFile)
    os.remove(outFile)
    return COILS, HOTLOOPS, REM465

def getSlices(NNdata, fold, join_frame, peak_frame, expect_val):
    slices = []
    inSlice = 0
    for i in range(len(NNdata)):
        if inSlice:
            if NNdata[i] < expect_val:
                if maxSlice >= fold*expect_val:
                    slices.append([beginSlice, endSlice])
                inSlice = 0
            else:
                endSlice += 1
                if NNdata[i] > maxSlice:
                    maxSlice = NNdata[i]
        elif NNdata[i] >= expect_val:
            beginSlice = i
            endSlice = i
            inSlice = 1
            maxSlice = NNdata[i]
    if inSlice and maxSlice >= fold*expect_val:
        slices.append([beginSlice, endSlice])

    i = 0
    while i < len(slices):
        if i+1 < len(slices) and slices[i+1][0]-slices[i][1] <= join_frame:
            slices[i] = [ slices[i][0], slices[i+1][1] ]
            del slices[i+1]
        elif slices[i][1]-slices[i][0]+1 < peak_frame:
            del slices[i]
        else:
            i += 1
    return slices


def reportSlicesTXT(slices, sequence):
    if slices == []:
        s = sequence.lower()
    else:
        if slices[0][0] > 0:
            s = (sequence[0:slices[0][0]]).lower()
        else:
            s = ''
        for i in range(len(slices)):
            if i > 0:
                sys.stdout.write(', ')
            sys.stdout.write( str(slices[i][0]+1) + '-' + str(slices[i][1]+1) )
            s = s + (sequence[slices[i][0]:(slices[i][1]+1)]).upper()
            if i < len(slices)-1:
                s = s + (sequence[(slices[i][1]+1):(slices[i+1][0])]).lower()
            elif slices[i][1] < len(sequence)-1:
                s = s + (sequence[(slices[i][1]+1):(len(sequence))]).lower()
    print ('')
    print (s)

def new_smooth(window,derivative,predicted_raw):
    if len(predicted_raw) < 2*window:
        window = len(predicted_raw)/2
    elif window == 0:
        window = 1
    outFile = tempfile.mktemp()
    inFile= tempfile.mktemp() 
    #inFile = 'oc1_test.txt'
    #outFile ='oc1_test_result.txt'
    with open(inFile, 'w') as filehandle:
        for e in predicted_raw:
            filehandle.write('%s\n' % e)   
    system_call = SG_bin + ' -V0 -D' + str(derivative) + ' -n' + str(window)+','+str(window) + ' '+ inFile + ' >' + outFile
    
    system(system_call)
    SG_results = []
    resultsFile = open(outFile,'r')
    results = resultsFile.readlines()
    resultsFile.close()
    
    for result in results:         
        temp = round(float(result),6)
        SG_results.append(temp)    
    os.remove(outFile)
    os.remove(inFile)
    return SG_results

In [4]:
smooth_frame = 8
peak_frame = 8
join_frame = 4
fold_coils = 1.4
fold_hotloops = 1.2
fold_rem465 = 1.2
sequence = 'STVSDKFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDE'

In [10]:
COILS_raw, HOTLOOPS_raw, REM465_raw = JensenNet(sequence)
COILS_smooth = new_smooth(smooth_frame,0,COILS_raw)
            # Run Savitzky-Golay
print(COILS_smooth,'\n\n', COILS_raw)

[0.839747, 0.785717, 0.760692, 0.826036, 0.827057, 0.844183, 0.845071, 0.859335, 0.822164, 0.822503, 0.818027, 0.817578, 0.815482, 0.801331, 0.802041, 0.789531, 0.776949, 0.744807, 0.709743, 0.670447, 0.64282, 0.614714, 0.595822, 0.577422, 0.567372, 0.574733, 0.578474, 0.585836, 0.599829, 0.598913, 0.61471, 0.593907, 0.626175, 0.487236, 0.579411, 0.642766, 0.472938, 0.622333, 0.572157, 0.824477] 

 [0.839747, 0.785717, 0.760692, 0.826036, 0.827057, 0.844183, 0.845071, 0.859335, 0.848351, 0.760752, 0.808663, 0.796088, 0.822037, 0.817223, 0.753782, 0.798229, 0.799795, 0.79531, 0.785854, 0.641484, 0.622113, 0.703105, 0.477405, 0.52035, 0.451666, 0.631818, 0.574987, 0.684282, 0.578679, 0.654651, 0.624316, 0.611977, 0.626175, 0.487236, 0.579411, 0.642766, 0.472938, 0.622333, 0.572157, 0.824477]
